In [90]:
import numpy as np
import pandas as pd
from pandas.api.types import is_string_dtype, is_numeric_dtype
import math
from sklearn.ensemble import RandomForestRegressor



In [91]:
PATH= '/home/halil/Belgeler/Kodlama calismalari/data/bulldozers'
df=pd.read_csv(f"{PATH}/Train.csv",low_memory=False,
parse_dates=["saledate"])

# Exercise

In [45]:
df

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,2006-11-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,2004-03-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2004-02-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,2011-05-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,2009-07-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401120,6333336,10500,1840702,21439,149,1.0,2005,NaN,NaN,2011-11-02,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401121,6333337,11000,1830472,21439,149,1.0,2005,NaN,NaN,2011-11-02,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401122,6333338,11500,1887659,21439,149,1.0,2005,NaN,NaN,2011-11-02,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401123,6333341,9000,1903570,21435,149,2.0,2005,NaN,NaN,2011-10-25,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN


In [92]:
df["UsageBand"]

0            Low
1            Low
2           High
3           High
4         Medium
           ...  
401120       NaN
401121       NaN
401122       NaN
401123       NaN
401124       NaN
Name: UsageBand, Length: 401125, dtype: object

In [93]:
for n,c in df.items():
    print(n)
    print("----------------------")
    print(c)
    break

SalesID
----------------------
0         1139246
1         1139248
2         1139249
3         1139251
4         1139253
           ...   
401120    6333336
401121    6333337
401122    6333338
401123    6333341
401124    6333342
Name: SalesID, Length: 401125, dtype: int64


In [96]:
# string dtype ı olan cloumbları pandas category tipine çevirme
def train_cats(df):
    for n,c in df.items():
        if is_string_dtype(c):
            df[n]=c.astype("category").cat.as_ordered()

In [95]:
def apply_cats(df,train):
    for n,c in df.items():
        if train[n].dtype =="category":
            df[n]=pd.Categorical(c,categories=train[n].cat.categories , ordered=True)

In [86]:
train_cats(df)

TypeError: apply_cats() missing 1 required positional argument: 'train'

In [97]:
df["UsageBand"]

0            Low
1            Low
2           High
3           High
4         Medium
           ...  
401120       NaN
401121       NaN
401122       NaN
401123       NaN
401124       NaN
Name: UsageBand, Length: 401125, dtype: object

In [98]:
df["SalePrice"].cat.categories

AttributeError: Can only use .cat accessor with a 'category' dtype

In [7]:
df["UsageBand"].cat.set_categories(["High","Medium","Low"],ordered=True,inplace=True)

/tmp/ipykernel_34687/1399414152.py:1: FutureWarning: The `inplace` parameter in pandas.Categorical.set_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  df["UsageBand"].cat.set_categories(["High","Medium","Low"],ordered=True,inplace=True)


In [8]:
df["UsageBand"].cat.categories

Index(['High', 'Medium', 'Low'], dtype='object')

In [28]:
df["UsageBand"].cat.codes

0         2
1         2
2         0
3         0
4         1
         ..
401120   -1
401121   -1
401122   -1
401123   -1
401124   -1
Length: 401125, dtype: int8

In [9]:
def numericalize(df,col,name):
    if not is_numeric_dtype(col):
        df[name] = col.cat.codes +1

In [32]:
numericalize(df,df["UsageBand"],"UsageBand")

In [39]:
df["UsageBand"]

0            Low
1            Low
2           High
3           High
4         Medium
           ...  
401120       NaN
401121       NaN
401122       NaN
401123       NaN
401124       NaN
Name: UsageBand, Length: 401125, dtype: object

# Datatime Column

In [4]:
df["saledate"]

0        2006-11-16
1        2004-03-26
2        2004-02-26
3        2011-05-19
4        2009-07-23
            ...    
401120   2011-11-02
401121   2011-11-02
401122   2011-11-02
401123   2011-10-25
401124   2011-10-25
Name: saledate, Length: 401125, dtype: datetime64[ns]

In [108]:
class a:
    x=2
b=a()
b.x

2

In [109]:
getattr(b,"x")

2

In [115]:
def add_datepart(df,dt_name,drop=True,time=False):
    dt_column=df[dt_name]
    column_dtype = dt_column.dtype

    attr=['year','month','week','day','dayofweek','dayofyear',
          'is_month_end','is_month_start','is_quarter_end','is_quarter_start','is_year_end','is_year_start']
    for a in attr:
        df["Date"+a.capitalize()] = getattr(dt_column.dt,a)      
    
    df["Data" + "Elapsed"] = dt_column.astype(np.int64)// 10 **9

    if drop:
        df.drop(dt_name,axis=1,inplace=True)

In [116]:
add_datepart(df,"saledate")

/tmp/ipykernel_34687/2308612856.py:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df["Date"+a.capitalize()] = getattr(dt_column.dt,a)


In [62]:
df

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,DateDay,DateDayofweek,DateDayofyear,DateIs_month_end,DateIs_month_start,DateIs_quarter_end,DateIs_quarter_start,DateIs_year_end,DateIs_year_start,DataElapsed
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,521D,...,16,3,320,False,False,False,False,False,False,1163635200
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,950FII,...,26,4,86,False,False,False,False,False,False,1080259200
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,226,...,26,3,57,False,False,False,False,False,False,1077753600
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,PC120-6E,...,19,3,139,False,False,False,False,False,False,1305763200
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,S175,...,23,3,204,False,False,False,False,False,False,1248307200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401120,6333336,10500,1840702,21439,149,1.0,2005,NaN,NaN,35NX2,...,2,2,306,False,False,False,False,False,False,1320192000
401121,6333337,11000,1830472,21439,149,1.0,2005,NaN,NaN,35NX2,...,2,2,306,False,False,False,False,False,False,1320192000
401122,6333338,11500,1887659,21439,149,1.0,2005,NaN,NaN,35NX2,...,2,2,306,False,False,False,False,False,False,1320192000
401123,6333341,9000,1903570,21435,149,2.0,2005,NaN,NaN,30NX,...,25,1,298,False,False,False,False,False,False,1319500800


# Missin Value

In [71]:
d = {"a":[1,2,3,4,4,np.nan],"b":[1,2,3,3,4,5]}
toy=pd.DataFrame(d)

In [72]:
toy

,a,b
0,1.0,1
1,2.0,2
2,3.0,3
3,4.0,3
4,4.0,4
5,NaN,5


In [73]:
toy.isnull()

,a,b
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
5,True,False


In [75]:
for a in toy:
    toy[f"{a.capitalize()}: non type"]=toy[f"{a}"].isnull()


In [76]:
toy

,a,b,A: non type,B: non type
0,1.0,1,False,False
1,2.0,2,False,False
2,3.0,3,False,False
3,4.0,3,False,False
4,4.0,4,False,False
5,NaN,5,True,False


In [77]:
def fix_missing(df,col,name):
    if is_numeric_dtype(col):
        if pd.isnull(col).sum:
            df[name+"_na"]=pd.isnull(col)
        df[name]=col.fillna(col.median())

In [78]:
for n,c in df.items():
    if is_numeric_dtype(c):
        if df[n].isnull().sum():
            print(n)

auctioneerID
MachineHoursCurrentMeter


In [81]:
print(df["auctioneerID"].isnull().sum())
print(df["MachineHoursCurrentMeter"].isnull().sum())


20136
258360


In [82]:
fix_missing(df,df["MachineHoursCurrentMeter"],"MachineHoursCurrentMeter")

In [83]:
df["MachineHoursCurrentMeter"].isnull().sum()

0

In [84]:
df["MachineHoursCurrentMeter_na"]

0         False
1         False
2         False
3         False
4         False
          ...  
401120     True
401121     True
401122     True
401123     True
401124     True
Name: MachineHoursCurrentMeter_na, Length: 401125, dtype: bool

# Prod_Df

In [41]:
"""def fix_missing(df,col,name):
    if is_numeric_dtype(col):
        if pd.isnull(col).sum:
            df[name+"_na"]=pd.isnull(col)
        df[name]=col.fillna(col.median())"""

'def fix_missing(df,col,name):\n    if is_numeric_dtype(col):\n        if pd.isnull(col).sum:\n            df[name+"_na"]=pd.isnull(col)\n        df[name]=col.fillna(col.median())'

In [120]:
def numericalize(df,col,name):
    if not is_numeric_dtype(col):
        df[name] = col.cat.codes +1

In [43]:
"""def proc_df(df,y_fld):
    y = df[y_fld].values
    df.drop([y_fld],axis=1,inplace=True)

    for n,c in df.items():
        fix_missing(df,c,n)
        numericalize(df,c,n)

    return df,y    """

'def proc_df(df,y_fld):\n    y = df[y_fld].values\n    df.drop([y_fld],axis=1,inplace=True)\n\n    for n,c in df.items():\n        fix_missing(df,c,n)\n        numericalize(df,c,n)\n\n    return df,y    '

In [54]:
# Kolonları gösteren kod.
df.columns 

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'saledate', 'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc',
       'fiModelSeries', 'fiModelDescriptor', 'ProductSize',
       'fiProductClassDesc', 'state', 'ProductGroup', 'ProductGroupDesc',
       'Drive_System', 'Enclosure', 'Forks', 'Pad_Type', 'Ride_Control',
       'Stick', 'Transmission', 'Turbocharged', 'Blade_Extension',
       'Blade_Width', 'Enclosure_Type', 'Engine_Horsepower', 'Hydraulics',
       'Pushblock', 'Ripper', 'Scarifier', 'Tip_Control', 'Tire_Size',
       'Coupler', 'Coupler_System', 'Grouser_Tracks', 'Hydraulics_Flow',
       'Track_Type', 'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb',
       'Pattern_Changer', 'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type',
       'Travel_Controls', 'Differential_Type', 'Steering_Controls'],
      dtype='object')

In [99]:
# traindeki for missin yaptığımızda oluşan df[name]+_na sütununun karmaşıklık çıkarmaması için oluşturularn
def fix_missing2(df,col,name,nan_dict,is_train): # traindeki oluian 
    if is_train:
        if is_numeric_dtype(col):
            if pd.isnull(col).sum:
                df[name+"_na"]=pd.isnull(col)
                nan_dict=col.median()
                df[name]=col.fillna(col.median())   

    else:
        if is_numeric_dtype(col):
            if name in nan_dict:
                df[name+"_na"]=pd.isnull(col)
                df[name]=col.fillna(nan_dict[name])
        else:
            #df[name]=col.fillna(df[name].median())


In [106]:
# traindeki for missin yaptığımızda oluşan df[name]+_na sütununun karmaşıklık çıkarmaması için oluşturularn
def proc_df2(df, y_fld,nan_dict=None,is_train=True):
    
    df=df.copy()
    y=df[y_fld].values

    df.drop(y_fld,axis=1,inplace=True)

    if nan_dict is None:
        nan_dict={}
    
    for n,c in df.items():
        fix_missing2(df,c,n,nan_dict,is_train)
    for n,c in df.items():
        numericalize(df,c,n)
    
    if is_train:
        return df,y,nan_dict

    return df,y

In [101]:
def slip_train_val(df,n):
    return df[:n].copy(),df[n:].copy()

In [102]:
n_valid =12_000 # same as kaggle's test set size
n_train=len(df)-n_valid
raw_train,raw_valid=slip_train_val(df,n_train)

In [119]:
x_train,y_train,nas=proc_df2(raw_train,'SalePrice')

AttributeError: Can only use .cat accessor with a 'category' dtype

In [110]:
x_valid,y_valid=proc_df2(raw_valid,'SalePrice',nan_dict=nas,is_train=False)

TypeError: could not convert string to float: 'Low'

In [83]:
numericalize(raw_valid,raw_valid["Undercarriage_Pad_Width"],"Undercarriage_Pad_Width")

AttributeError: 'Series' object has no attribute 'catagory'

In [114]:
df

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,2006-11-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,2004-03-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2004-02-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,2011-05-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,2009-07-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401120,6333336,10500,1840702,21439,149,1.0,2005,NaN,NaN,2011-11-02,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401121,6333337,11000,1830472,21439,149,1.0,2005,NaN,NaN,2011-11-02,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401122,6333338,11500,1887659,21439,149,1.0,2005,NaN,NaN,2011-11-02,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401123,6333341,9000,1903570,21435,149,2.0,2005,NaN,NaN,2011-10-25,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
